In [1]:
%load_ext autoreload
%autoreload 2
from src.data.processing import tokenize_dataset
import transformers
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments,DataCollatorForSeq2Seq, AutoTokenizer
import evaluate
import datasets
import numpy as np

z:\NLP_project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = datasets.load_dataset("Samsung/samsum")

z:\NLP_project\venv\lib\site-packages\datasets\load.py:1491: FutureWarning: The repository for Samsung/samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Samsung/samsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [26]:
MODEL_LOCAL = "./.output/t5-summarizer/results"#"google-t5/t5-small"
LOCAL = True
DEVICE = "cuda"
model1 = transformers.AutoModelForSeq2SeqLM.from_pretrained("lucapl/t5-summarizer-samsum",local_files_only=False, device_map=DEVICE)
#tokenizer = transformers.AutoTokenizer.from_pretrained(model_local,local_files_only=LOCAL, device_map=DEVICE)

z:\NLP_project\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
MODEL_NAME = "google-t5/t5-small"
model2 = transformers.AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME,local_files_only=False, device_map=DEVICE)
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME,local_files_only=False, device_map=DEVICE)

In [14]:
tokenized_dataset = tokenize_dataset(data, tokenizer, True,data_prefix="summarize: ")
tokenized_dataset.set_format(type="torch")

Max source length: 255


Map: 100%|██████████| 15551/15551 [00:00<00:00, 29793.46 examples/s]


Max target length: 45


Map: 100%|██████████| 818/818 [00:00<00:00, 4130.69 examples/s]


In [15]:
rouge = datasets.load_metric("rouge")
rouge


C:\Users\Lucas\AppData\Local\Temp\ipykernel_16612\2953629268.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = datasets.load_metric("rouge")
z:\NLP_project\venv\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [16]:
# training_args = Seq2SeqTrainingArguments(
#     output_dir=".output/t5-summarizer",
#     learning_rate=1e-3,
#     per_device_train_batch_size=64,
#     per_device_eval_batch_size=64,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
# 	remove_unused_columns=False 
# )

# data_collator = DataCollatorForSeq2Seq(tokenizer,model,
# 	return_tensors="pt",
# 	label_pad_token_id=-100,
# 	pad_to_multiple_of=8)

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     data_collator=data_collator
# )

# # Evaluate the model
# eval_results = trainer.predict(tokenized_dataset["test"].select(range(10)))

In [27]:
out1 = model1.generate(tokenized_dataset["validation"].select(range(25))["input_ids"].to("cuda"),max_new_tokens=100)
out2 = model2.generate(tokenized_dataset["validation"].select(range(25))["input_ids"].to("cuda"),max_new_tokens=100)

In [28]:
n = 10
print("Dialogue:",data["validation"][n]["dialogue"])
print()
print("t5 finetuned:",tokenizer.decode(out1[n],skip_special_tokens=True))
print()
print("t5:",tokenizer.decode(out2[n],skip_special_tokens=True))
print()
print("expected:",data["validation"][n]["summary"])

Dialogue: Laura: I need a new printer :/
Laura: thinking about this one
Laura: <file_other>
Jamie: you're sure you need a new one?
Jamie: I mean you can buy a second hand one
Laura: could be

t5 finetuned: Laura needs a new printer. Jamie is sure she needs a new printer.

t5: Laura: you're sure you need a new one Laura: file_other> Jamie: you're sure you need a new one?

expected: Laura is going to buy a printer.


In [ ]:
#model1.push_to_hub("t5-summarizer-samsum")